In [1]:
from dotenv import load_dotenv

load_dotenv(verbose=True)

True

In [2]:
import weaviate
from weaviate.classes.init import Auth
import os

weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# headers = {
#     # Provide your required API key(s), e.g. Cohere, OpenAI, etc. for the configured vectorizer(s)
#     "X-INFERENCE-PROVIDER-API-KEY": os.environ["YOUR_INFERENCE_PROVIDER_KEY"],
# }

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    # headers=headers,
)

print(client.is_ready())

True


In [12]:
from datasets import load_dataset
import os

ds = load_dataset("hotchpotch/JQaRA", split="test")
N = int(os.getenv("LIMIT", "1000"))
subset = ds.select(range(min(N, len(ds))))

/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
CLASS_NAME = "JQaRA"

if client.collections.exists(CLASS_NAME):
    client.collections.delete(CLASS_NAME)

In [14]:
from weaviate.classes.config import Configure, Property, DataType

# OLLAMA_ENDPOINT="http://host.docker.internal:11434"
# EMBED_MODEL="nomic-embed-text"

client.collections.create(
    name=CLASS_NAME,
    vector_config=Configure.Vectors.text2vec_weaviate(),
    properties=[
        Property(name="q_id", data_type=DataType.TEXT),
        Property(name="title", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
    ],
)


In [15]:
from tqdm import tqdm

col = client.collections.get(CLASS_NAME)

with col.batch.dynamic() as batch:
    for row in tqdm(subset):
        batch.add_object(
            properties={
                "q_id": row.get("q_id"),
                "title": row.get("title"),
                "text": row.get("text"),
            }
        )

print(f"Imported {subset.num_rows} objects into {CLASS_NAME}")

100%|██████████| 1000/1000 [00:00<00:00, 6356.34it/s]


Imported 1000 objects into JQaRA


In [16]:
from weaviate.agents.query import QueryAgent

# Instantiate a new agent object
qa = QueryAgent(
    client=client, collections=["JQaRA", "BLOG_cloud"]
)

In [32]:
response = qa.ask("このデータセットはなんですか？")

response.display()

╭───────────────────────────────────────────── 💬 Ask Mode Response ──────────────────────────────────────────────╮
│                                                                                                                 │
│ このデータセットは、Vespaというオープンソースの検索エンジンプラットフォーム上で扱うドキュメント群の一例です。具 │
│ 体的には「job（求人票）」というスキーマで定義されており、タイトル（title）や詳細（description）を持つ求人情報が │
│ 複数含まれています。このセットには、タイトルや説明文が異なる複数の求人文書が登録されており、検索結果の重複排除  │
│ （dedupe）を試すために使われています。                                                                          │
│                                                                                                                 │
│ このデータセットは、Vespaでの検索の重複排除機能を説明・検証するために用いられています。検索結果に同じタイトルの │
│ 求人が複数ある場合、それらをグループ化して重複をまとめ、代表する1件のみを結果として返す仕組みを実演可能です。   │
│                                                                                                                 │
│ 簡単に言えば、このデータセットはVespa検索エンジンの動作検証用の求人情報ドキュメント群で、重複する情報を効率的に │
│ 整理・表示する方法を学ぶための例となっています。                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 1/2 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='データセット 説明 概要 collection 概要 内容',                                                        │
│     filters=None,                                                                                               │
│     collection='JQaRA',                                                                                         │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 2/2 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='データセットの説明 概要 どんなデータセットか このデータセットについて',                              │
│     filters=None,                                                                                               │
│     collection='BLOG_cloud',                                                                                    │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='ec6c8fe4-a018-491f-819a-fb861b8f62b0' collection='BLOG_cloud'                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📊 Usage Statistics        
┌──────────────────────────┬──────┐
│ Model Units:             │ 367  │
│ Usage in Plan:           │ True │
│ Remaining Plan Requests: │ 976  │
└──────────────────────────┴──────┘

Total Time Taken: 11.36s

In [9]:
print(subset[0]["question"])

摂氏ではマイナス273.15度にあたる、全ての原子の振動が停止する最も低い温度を何というでしょう?


In [7]:
question = subset[0]["question"]

response = qa.ask(question)

response.display()

╭───────────────────────────────────────────── 💬 Ask Mode Response ──────────────────────────────────────────────╮
│                                                                                                                 │
│ 摂氏−273.15度にあたり、全ての原子の振動が理論上停止する最も低い温度は「絶対零度（ぜったいれいど）」と呼ばれます │
│ 。絶対零度は熱力学上の最低温度であり、理想気体のエントロピーやエンタルピーが最低値となる0ケルビン（K）に相当し  │
│ ます。この温度では原子の熱振動（原子の乱雑な並進運動エネルギー）が停止した状態とされますが、量子力学の不確定性  │
│ 原理により、完全に振動が止まるわけではなく、零点振動と呼ばれる最低エネルギーの振動は残ります。有限の操作で絶対  │
│ 零度に達することはできないともされています。                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 1/1 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='摂氏-273.15度 全ての原子の振動が停止する 最も低い温度 名称',                                         │
│     filters=None,                                                                                               │
│     collection='JQaRA_cloud',                                                                                   │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='57159308-a0d9-4a1c-a095-1bff32dc510d' collection='JQaRA_cloud'                                    │
│  - object_id='e0f8dea8-588c-49c9-9fd7-223b2cf4bf8e' collection='JQaRA_cloud'                                    │
│  - object_id='3180d8b8-1dd0-45ca-ae1f-3e2318b26faa' collection='JQaRA_cloud'                                    │
│  - object_id='7b6ff162-45f3-4a49-88db-16527d7bd7a0' collection='JQaRA_cloud'                                    │
│  - object_id='2a075558-3008-401a-914a-20387bb1c4b1' collection='JQaRA_cloud'                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📊 Usage Statistics        
┌──────────────────────────┬──────┐
│ Model Units:             │ 250  │
│ Usage in Plan:           │ True │
│ Remaining Plan Requests: │ 992  │
└──────────────────────────┴──────┘

Total Time Taken: 9.18s

In [8]:
print("\n=== Query Agent Response ===")
print(f"Original Query: {response.searches[0].query}\n")

print("🔍 Final Answer Found:")
print(f"{response.final_answer}\n")

print("🔍 Searches Executed:")
for collection_searches in response.searches:
    for result in collection_searches:
        print(f"- {result}\n")

if len(response.aggregations) > 0:
    print("📊 Aggregation Results:")
    for collection_aggs in response.aggregations:
        for agg in collection_aggs:
            print(f"- {agg}\n")

if response.missing_information:
    if response.is_partial_answer:
        print("⚠️ Answer is Partial - Missing Information:")
    else:
        print("⚠️ Missing Information:")
    for missing in response.missing_information:
        print(f"- {missing}")


=== Query Agent Response ===
Original Query: 摂氏-273.15度 全ての原子の振動が停止する 最も低い温度 名称

🔍 Final Answer Found:
摂氏−273.15度にあたり、全ての原子の振動が理論上停止する最も低い温度は「絶対零度（ぜったいれいど）」と呼ばれます。絶対零度は熱力学上の最低温度であり、理想気体のエントロピーやエンタルピーが最低値となる0ケルビン（K）に相当します。この温度では原子の熱振動（原子の乱雑な並進運動エネルギー）が停止した状態とされますが、量子力学の不確定性原理により、完全に振動が止まるわけではなく、零点振動と呼ばれる最低エネルギーの振動は残ります。有限の操作で絶対零度に達することはできないともされています。

🔍 Searches Executed:
- ('query', '摂氏-273.15度 全ての原子の振動が停止する 最も低い温度 名称')

- ('filters', None)

- ('collection', 'JQaRA_cloud')

- ('sort_property', None)



In [6]:


import json
from pathlib import Path

DATA_DIR = Path("data")
BLOG_JSON = DATA_DIR / "blog.json"

with open(BLOG_JSON, "r", encoding="utf-8") as f:
    items = json.load(f)

# 各Markdownを読み込む（title, path, content）
docs = []
for it in items:
    title = it.get("title")
    rel_path = it.get("path")
    md_path = DATA_DIR / rel_path
    if not md_path.exists():
        raise FileNotFoundError(f"Markdown が見つかりません: {md_path}")
    content = md_path.read_text(encoding="utf-8")
    docs.append({
        "title": title,
        "path": str(rel_path),
        "content": content,
    })

print(f"Loaded {len(docs)} docs from {BLOG_JSON}")

Loaded 4 docs from data/blog.json


In [8]:
docs[3]

{'title': 'WeaviateでAgentを利用した検索',
 'path': 'takatori_blog_4.md',
 'content': '# WeaviateでAgentを利用した検索\n\nLayerXのAi Workforce事業部で検索エンジニアをしている鷹取(@takatorisatoshi)です。\n\nこの記事はLayerX AIエージェント ブログリレー 43日目の記事です。\n昨日は[@po3rin](https://x.com/po3rin)さんの[Repeated Samplingを使ったLLM推論時スケーリングで麻雀点数計算問題生成タスクを解くぞ！](https://tech.layerx.co.jp/entry/mahjong-repeated-sampling)でした。\n\nこの記事ではベクトルデータベースである[Weaviate](https://weaviate.io/)を使って、Agentを使った高度な検索を簡単に実現する方法を紹介します。\n\n## はじめに\n\nLLMの普及により、検索のワークロードが大きく変わりつつあります。\n従来は人間のユーザが検索をしていましたが、エージェントも検索するようになり、長く複雑なクエリが発行さるようになりました。\n<!-- チャット形式で自然言語の質問(質問応答)の検索が大幅に増えています。 -->\nこれにより、通常のRAGでは回答できないような質問も検索する必要が出てきました。\n\nhornet.dev\n> 従来の検索インフラは、人間というたった一人のユーザー向けに調整されていました。スタックは、短いキーワードクエリ、ミリ秒単位の応答時間、そして上位10件のスニペットに最適化されていました。しかし、現在ではエージェントが検索を実行するようになり、ワークロードが変化しています。エージェントは推論ループ内で長く構造化されたクエリを発行します。そして、ドキュメント全体を読み取り、レイテンシと引き換えにスループットと再現率を向上させています。こうしたワークロードを従来の検索インフラに無理やり押し付けると、コストが上昇し、結果の質が低下します。\n\nそこで、Agentic RAGのような、検索にAgentを組み込んだ手法が提案されています。\n\nhttps://ar

In [5]:
from weaviate.classes.config import Configure, Property, DataType

CLASS_NAME = "BLOG_cloud"
# OLLAMA_ENDPOINT="http://host.docker.internal:11434"
# EMBED_MODEL="nomic-embed-text"

client.collections.create(
    name=CLASS_NAME,
    vector_config=Configure.Vectors.text2vec_weaviate(),
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="path",    data_type=DataType.TEXT),
        Property(name="content", data_type=DataType.TEXT),
    ],
)


UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name BLOG_cloud already exists'}]}.

In [10]:
from tqdm import tqdm

col = client.collections.get(CLASS_NAME)

with col.batch.dynamic() as batch:
    for row in tqdm([docs[3]]):
        batch.add_object(
            properties={
                "title": row.get("title"),
                "path": row.get("path"),
                "content": row.get("content"),
            }
        )

100%|██████████| 1/1 [00:00<00:00, 5269.23it/s]


NameError: name 'subset' is not defined

In [3]:
from weaviate.agents.query import QueryAgent

# Instantiate a new agent object
qa = QueryAgent(
    client=client, collections=["JQaRA_cloud", "BLOG_cloud"]
)

In [19]:
response = qa.ask("Vespaで検索結果のdedupeを行う方法は？")

response.display()

╭───────────────────────────────────────────── 💬 Ask Mode Response ──────────────────────────────────────────────╮
│                                                                                                                 │
│ Vespaで検索結果の重複排除（dedupe）を行うには、Vespaの検索結果集計機能である「grouping」を使います。具体的には  │
│ 、重複判定のキー（たとえば記事のタイトルなど）を指定して、そのキーでグループ化し、各グループから上位1件のみを結 │
│ 果に残す方法です。                                                                                              │
│                                                                                                                 │
│ 例として、`title`フィールドを重複キー（grouping_key）としてdedupeする場合のクエリ構造は以下のようになります。   │
│                                                                                                                 │
│ ```                                                                                                             │
│ select * from job where 条件 | all(                                                                             │
│   group(title) max(200) each(                                                                                   │
│     max(1)                                                                                                      │
│     output(count())                                                                                             │
│     each(output(summary()))                                                                                     │
│   )                                                                                                             │
│ )                                                                                                               │
│ ```                                                                                                             │
│                                                                                                                 │
│ 上記の意味は、                                                                                                  │
│                                                                                                                 │
│ - 検索結果全体を`all`でラップし、                                                                               │
│ - `group(title)`でtitleごとにグルーピング、                                                                     │
│ - `max(200)`で上位200グループまで絞り、                                                                         │
│ - 各グループから`max(1)`で1件だけ抽出し、                                                                       │
│ - 各グループのドキュメント数を`output(count())`で取得、                                                         │
│ - そのドキュメントの詳細を`output(summary())`で取得                                                             │
│                                                                                                                 │
│ という流れです。                                                                                                │
│                                                                                                                 │
│ このグルーピングによる重複排除を使うことで、タイトルが同じ複数のドキュメントから最上位の1件のみを検索結果として │
│ 返し、重複結果を排除できます。                                                                                  │
│                                                                                                                 │
│ **注意点として：**                                                                                              │
│                                                                                                                 │
│ - この方法では検索結果の形が通常のドキュメントリストと異なり、grouping特有のネスト構造のレスポンスになります。  │
│ - groupのカウントはHyperLogLog++アルゴリズムによる推定値であり、正確な数とは限りません。                        │
│ - ページネーションはcontinuationを用いるため、offset指定に対応していないため扱いに注意が必要です。              │
│                                                                                                                 │
│ また、ローカル環境でのVespaセットアップやドキュメント登録も含めて、dedup

╭───────────────────────────────────────────────── 🔭 Search 1/1 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='Vespa 検索結果 dedupe 重複排除 方法',                                                                │
│     filters=None,                                                                                               │
│     collection='BLOG_cloud',                                                                                    │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='ec6c8fe4-a018-491f-819a-fb861b8f62b0' collection='BLOG_cloud'                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📊 Usage Statistics        
┌──────────────────────────┬──────┐
│ Model Units:             │ 259  │
│ Usage in Plan:           │ True │
│ Remaining Plan Requests: │ 988  │
└──────────────────────────┴──────┘

Total Time Taken: 11.30s

In [22]:
print("\n=== Query Agent Response ===")
print(f"Query: {response.searches[0].query}\n")

print("🔍 Final Answer Found:")
print(f"{response.final_answer}\n")


print("🔍 Searches Executed:")
for collection_searches in response.searches:
    for result in collection_searches:
        print(f"# {result}\n")

if len(response.aggregations) > 0:
    print("📊 Aggregation Results:")
    for collection_aggs in response.aggregations:
        for agg in collection_aggs:
            print(f"# {agg}\n")

if response.missing_information:
    if response.is_partial_answer:
        print("⚠️ Answer is Partial - Missing Information:")
    else:
        print("⚠️ Missing Information:")
    for missing in response.missing_information:
        print(f"# {missing}")


=== Query Agent Response ===
Query: Vespa 検索結果 dedupe 重複排除 方法

🔍 Final Answer Found:
Vespaで検索結果の重複排除（dedupe）を行うには、Vespaの検索結果集計機能である「grouping」を使います。具体的には、重複判定のキー（たとえば記事のタイトルなど）を指定して、そのキーでグループ化し、各グループから上位1件のみを結果に残す方法です。

例として、`title`フィールドを重複キー（grouping_key）としてdedupeする場合のクエリ構造は以下のようになります。

```
select * from job where 条件 | all(
  group(title) max(200) each(
    max(1)
    output(count())
    each(output(summary()))
  )
)
```

上記の意味は、

- 検索結果全体を`all`でラップし、
- `group(title)`でtitleごとにグルーピング、
- `max(200)`で上位200グループまで絞り、
- 各グループから`max(1)`で1件だけ抽出し、
- 各グループのドキュメント数を`output(count())`で取得、
- そのドキュメントの詳細を`output(summary())`で取得

という流れです。

このグルーピングによる重複排除を使うことで、タイトルが同じ複数のドキュメントから最上位の1件のみを検索結果として返し、重複結果を排除できます。

**注意点として：**

- この方法では検索結果の形が通常のドキュメントリストと異なり、grouping特有のネスト構造のレスポンスになります。
- groupのカウントはHyperLogLog++アルゴリズムによる推定値であり、正確な数とは限りません。
- ページネーションはcontinuationを用いるため、offset指定に対応していないため扱いに注意が必要です。

また、ローカル環境でのVespaセットアップやドキュメント登録も含めて、dedupe検証が可能です。Vespa CLIやDockerでの環境構築、application package構成（services.xm

In [34]:
question = subset[200]["question"]

print(question)

response = qa.ask(question)

response.display()

わざと人に逆らう言動をする人を、民話に出てくる悪い鬼に例えて何というでしょう?


╭───────────────────────────────────────────── 💬 Ask Mode Response ──────────────────────────────────────────────╮
│                                                                                                                 │
│ わざと人に逆らう言動をする人を民話に出てくる悪い鬼に例える場合、「天邪鬼（あまのじゃく）」という表現が使われま  │
│ す。天邪鬼はもともと「人の心を見計らって悪戯をしかける子鬼」とされ、現代では「他者の思想や言動に逆らうひねくれ  │
│ 者」として比喩的に用いられています。つまり、人に意地悪をしたり、わざと反抗的な態度を取る人を指す言葉として天邪  │
│ 鬼が適しています。                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 1/1 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='わざと人に逆らう言動をする人 民話 悪い鬼 例える 表現',                                               │
│     filters=None,                                                                                               │
│     collection='JQaRA',                                                                                         │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='4484a19c-75a4-4534-9eca-558d0c04ac79' collection='JQaRA'                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📊 Usage Statistics        
┌──────────────────────────┬──────┐
│ Model Units:             │ 159  │
│ Usage in Plan:           │ True │
│ Remaining Plan Requests: │ 968  │
└──────────────────────────┴──────┘

Total Time Taken: 7.61s

In [35]:
response = qa.ask("Vespaって何？")

response.display()

╭───────────────────────────────────────────── 💬 Ask Mode Response ──────────────────────────────────────────────╮
│                                                                                                                 │
│ Vespa（ヴェスパ）とは、Yahooが開発したオープンソースの検索エンジンプラットフォームです。大規模データセットの高  │
│ 速でスケーラブルな検索や探索を実現するために分散処理技術を活用しており、多様なデータタイプに対応しています。    │
│                                                                                                                 │
│ 主な特徴として、Vespaは以下のような機能を持っています。                                                         │
│                                                                                                                 │
│ 1. **高度な検索エンジン機能**                                                                                   │
│    テキスト検索のほか、機械学習モデルを組み込んだランキング（リランキング）が標準でサポートされており、検索結果 │
│ の精度向上に活用できます。                                                                                      │
│    例えば、独自のrank-profile設定や機械学習モデル（XGBoostなど）をVespaにデプロイして、検索結果の並び替えを改善 │
│ することが可能です。                                                                                            │
│                                                                                                                 │
│ 2. **重複排除（dedupe）機能**                                                                                   │
│    検索結果から重複する情報を取り除きユーザー体験の向上を図れます。Vespaのグルーピング（grouping）機能を使い、  │
│ 同じ特定キーで分けられたドキュメントの中から代表的な1件だけを検索結果に残すことができます。                     │
│                                                                                                                 │
│ 3. **豊富なデプロイ・管理ツール**                                                                               │
│    Dockerコンテナで動かせ、CLIツールが提供されているためローカル環境で簡単にセットアップ・操作が可能です。設定  │
│ ・スキーマ定義を一式デプロイしてサービスを構築します。                                                          │
│                                                                                                                 │
│ まとめると、VespaはYahooが開発した大規模検索エンジン向けの分散プラットフォームであり、高速な全文検索や機械学習  │
│ モデルによる検索ランキングの改善、検索結果の重複排除など、多彩な検索関連機能を備えています。オープンソースで提  │
│ 供されているため、企業や開発者が自由に利用・カスタマイズして活用できるシステムです。                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 1/1 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='Vespaとは何か 概要 説明',                                                                            │
│     filters=None,                                                                                               │
│     collection='BLOG_cloud',                                                                                    │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='ec6c8fe4-a018-491f-819a-fb861b8f62b0' collection='BLOG_cloud'                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📊 Usage Statistics        
┌──────────────────────────┬──────┐
│ Model Units:             │ 386  │
│ Usage in Plan:           │ True │
│ Remaining Plan Requests: │ 964  │
└──────────────────────────┴──────┘

Total Time Taken: 11.49s

In [36]:
response = qa.ask("ElasticsearchとVespaの違いは？")

response.display()

╭───────────────────────────────────────────── 💬 Ask Mode Response ──────────────────────────────────────────────╮
│                                                                                                                 │
│ ElasticsearchとVespaはどちらも検索エンジンとして利用されるオープンソースのプラットフォームですが、いくつかの違  │
│ いがあります。                                                                                                  │
│                                                                                                                 │
│ 1. **開発元と特徴**                                                                                             │
│    - **Elasticsearch**: Apache                                                                                  │
│ Luceneをベースに開発されており、広く使われている全文検索エンジンです。最近ではANN（近似最近傍探索）をサポートし │
│ 、機械学習を利用したベクトル検索（Semantic                                                                      │
│ Search）にも対応しつつあります。Elasticsearchは大規模分散環境に強く、多種多様なデータの全文検索や分析に使用され │
│ ています。                                                                                                      │
│    - **Vespa**:                                                                                                 │
│ Yahooが開発したオープンソースの検索プラットフォームで、高速な検索・推薦エンジンとして設計されています。検索ラン │
│ キングの機械学習モデルの組込みやリランキングをデフォルトで強力にサポートしており、プラグイン不要でXGBoostベース │
│ のモデルを利用可能。また、検索結果の重複排除（dedupe）など、検索の多機能な制御も可能です。                      │
│                                                                                                                 │
│ 2. **機械学習モデルの統合**                                                                                     │
│    - **Elasticsearch**は、Lucene                                                                                │
│ 9以降のANN対応を背景にベクトル検索が強化されていますが、機械学習モデルの直接的な検索ランキングへの組込みについ  │
│ てはElasticsearch本体にそのままの形でのサポートは限定的（将来的に拡充予定）。                                   │
│    -                                                                                                            │
│ **Vespa**は検索ランキングへの機械学習モデル組込みが標準機能で、独自のrank-profile設定で特徴量取得からモデルのデ │
│ プロイ、検索時のリランキングまで一連の機械学習ランキング機能が整備されています。                                │
│                                                                                                                 │
│ 3. **検索結果の集計・制御機能**                                                                                 │
│    -                                                                                                            │
│ **Vespa**は検索結果のグルーピングや重複排除（dedupe）がグループ化関数（grouping）によって柔軟に実装可能。複雑な │
│ 集計や重複排除のロジックを検索クエリ内で記述できる点が特徴です。                                                │
│    -                                                                                                            │
│ **Elasticsearch**も豊富な集約機能を持つが、Vespaほどの複雑な検索クエリ内のグルーピング操作とはやや異なる設計。  │
│                                                                                                                 │
│ 4. **セットアップと運用**                                                                                       │
│    -                                                                                                            │
│ 両者ともDockerやCLIツールによるローカル環境構築が可能ですが、Vespaは公式にDevContainerやVispana（GUIツール）も  │
│ 提供され、開発～運用までの体験が比較的充実しています。                                                          │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ まとめると、Elasticsearchは大規模な全文検索や近似最近傍検索（ANN）に強みがあり、Luceneベースで幅広く使われてい  │
│ ます。一方で、Vespaは検索に機械学習モデルを密接に組込みたい場合や、検索結果の高度な集計・重複排除機能を求めるケ │
│ ースで特に優れています。用途や求める機能によって選択が分かれるでしょう。                                        │
│                                                                                         

╭───────────────────────────────────────────────── 🔭 Search 1/1 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='ElasticsearchとVespaの違い 比較 特徴 メリット デメリット',                                           │
│     filters=None,                                                                                               │
│     collection='BLOG_cloud',                                                                                    │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='066850da-213e-4bd6-88ef-9558f6c97543' collection='BLOG_cloud'                                     │
│  - object_id='ec6c8fe4-a018-491f-819a-fb861b8f62b0' collection='BLOG_cloud'                                     │
│  - object_id='014752fa-9d01-4136-a840-c6ee5a57944b' collection='BLOG_cloud'                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📊 Usage Statistics        
┌──────────────────────────┬──────┐
│ Model Units:             │ 396  │
│ Usage in Plan:           │ True │
│ Remaining Plan Requests: │ 960  │
└──────────────────────────┴──────┘

Total Time Taken: 16.90s

In [26]:
search_response = qa.search("絶対零度とは？")

print(f"Query Agentが発行したquery:{search_response.searches[0].query}")
print(f"Query Agentが検索したcollection:{search_response.searches[0].collection}")

for obj in search_response.search_results.objects:
    print(f"id:{obj.properties["q_id"]}, explan_score:{obj.metadata["explain_score"]}")

Query Agentが発行したquery:絶対零度とは 温度の定義 基本概念 絶対温度 ゼロケルビン 極低温の物理的意味
Query Agentが検索したcollection:JQaRA
id:QA20CAPR-1004, explan_score:
Hybrid (Result Set vector,hybridVector) Document 729a82ae-fa6f-498b-9dcd-d19b2fe02494: original score 0.7352022, normalized score: 0.68195397
id:QA20CAPR-1004, explan_score:
Hybrid (Result Set vector,hybridVector) Document 26b007dd-2714-4dcd-8169-ead737cd7b80: original score 0.74868655, normalized score: 0.7
id:QA20CAPR-1004, explan_score:
Hybrid (Result Set vector,hybridVector) Document 63a24114-0a5d-47fe-8bcf-e8959f0fe674: original score 0.6343899, normalized score: 0.5470378
id:QA20CAPR-1004, explan_score:
Hybrid (Result Set vector,hybridVector) Document 7f3689e9-81ff-4f37-aefb-4c68463c2c83: original score 0.61184263, normalized score: 0.516863
id:QA20CAPR-1004, explan_score:
Hybrid (Result Set vector,hybridVector) Document efc3cbec-e2fb-49de-98e4-f96fa3971180: original score 0.55024755, normalized score: 0.43443087
id:QA20CAPR-1004, explan_score:
Hybrid (Re

In [17]:
response = qa.ask("Weaviateって何？Vespaとの違いは？")

response.display()

╭───────────────────────────────────────────── 💬 Ask Mode Response ──────────────────────────────────────────────╮
│                                                                                                                 │
│ WeaviateとVespaはどちらも検索やデータ探索に使われる技術ですが、特徴や目的に違いがあります。                     │
│                                                                                                                 │
│ 【Weaviateとは】                                                                                                │
│ - オープンソースのベクトルデータベースで、Go言語で実装。                                                        │
│ - ベクトル検索を中心に、自然言語による高度な検索をエージェント（Query Agent）を使って簡単に実装可能。           │
│ - LLM（大規模言語モデル）と連携し、自然言語クエリを解析・実行し、回答を生成する仕組みが提供されている。         │
│ - クラウドでも自ホストでも利用でき、テキストの類似検索や質問応答などAIを活用した先進的な検索用途に強みがある。  │
│                                                                                                                 │
│ 【Vespaとは】                                                                                                   │
│ - Yahooが開発したオープンソースの検索エンジンプラットフォーム。                                                 │
│ - 大規模データセットの高速・スケーラブルな探索が可能。                                                          │
│ - 機械学習モデルを用いた検索ランキングのリランキング機能や、検索結果の重複排除（dedupe）を高度に制御できる。    │
│ - Dockerなどでローカル開発環境を構築しやすく、検索インフラとして柔軟に運用可能。                                │
│ - 多種多様なデータタイプに対応し、構造化されたスキーマとrankprofile設定により細かいチューニングができる。       │
│                                                                                                                 │
│ 【違いのまとめ】                                                                                                │
│ -                                                                                                               │
│ Weaviateはベクトル検索を核とし、自然言語クエリやエージェントを利用したAI連携が得意で、新しい検索体験を少ないコ  │
│ ードで実現可能。                                                                                                │
│ -                                                                                                               │
│ Vespaは伝統的かつ大規模な検索エンジンプラットフォームで、機械学習を利用したランキングや重複除外など高度な検索処 │
│ 理を組み込みやすい。                                                                                            │
│ -                                                                                                               │
│ 用途としては、WeaviateはAI活用したベクトル類似検索やQ&Aシステムに、VespaはスケーラブルなWeb検索や複雑なランキン │
│ グ制御が必要なシステムに適している。                                                                            │
│                                                                                                                 │
│ このように、WeaviateはAI時代の自然言語ベクトル検索に特化し使いやすさを重視、一方Vespaは構造化データや機械学習リ │
│ ランキングを使った精緻な検索基盤としての役割が大きいと言えます。                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 1/3 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='Weaviate vector database description features vector search agent usage',                            │
│     filters=None,                                                                                               │
│     collection='BLOG_cloud',                                                                                    │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 2/3 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='Vespa search engine description features machine learning ranking improvement',                      │
│     filters=None,                                                                                               │
│     collection='BLOG_cloud',                                                                                    │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── 🔭 Search 3/3 ─────────────────────────────────────────────────╮
│                                                                                                                 │
│ QueryResultWithCollectionNormalized(                                                                            │
│     query='WeaviateとVespaの違い 利用方法 技術 面 比較 用途',                                                   │
│     filters=None,                                                                                               │
│     collection='BLOG_cloud',                                                                                    │
│     sort_property=None                                                                                          │
│ )                                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 📊 No Aggregations Run                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── 📚 Sources ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  - object_id='97d062d1-abe5-4264-84e1-ad45bc6744fd' collection='BLOG_cloud'                                     │
│  - object_id='066850da-213e-4bd6-88ef-9558f6c97543' collection='BLOG_cloud'                                     │
│  - object_id='ec6c8fe4-a018-491f-819a-fb861b8f62b0' collection='BLOG_cloud'                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📊 Usage Statistics        
┌──────────────────────────┬──────┐
│ Model Units:             │ 617  │
│ Usage in Plan:           │ True │
│ Remaining Plan Requests: │ 951  │
└──────────────────────────┴──────┘

Total Time Taken: 14.63s